In [1]:
from lxml import etree
import glob

In [2]:
from pathlib import Path
from tqdm import tqdm

In [3]:
import pandas as pd
import numpy as np
import os,sys

In [4]:
import re
from PIL import Image

### 接口

In [5]:
def voc2yolo():
    center_x = (xmin + (xmax-xmin)/2)/width
    center_y = (ymin + (ymax-ymin)/2)/height
    bdb_w = (xmax-xmin)/width
    bdb_h = (ymax-ymin)/height

In [6]:
def parse_xml(p):
    trees = etree.parse(p)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    bboxes = []
    for obj in trees.findall('object'):
        bx = []
        bx.append(p)
        bx.append(int(width))
        bx.append(int(height))
        bdb = obj.find('bndbox')
        bx.append(obj.find('name').text)
        bx.append(int(bdb.find('xmin').text))
        bx.append(int(bdb.find('ymin').text))
        bx.append(int(bdb.find('xmax').text))
        bx.append(int(bdb.find('ymax').text))
        bboxes.append(bx)
    return bboxes

In [7]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}
model_label_map8 = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装_短款
    3: 5,  # 裤子_短款
    4: 6,  # 裙子
    5: 7,  # 连体装
    6: 4,  # 上装_长款
    7: 5,  # 裤子_长款
}

In [8]:
chi_label2id8 = {
    0: '鞋子',
    1: '包包',
    2: '上装_短款',
    3: '裤子_短款',
    4: '裙子',
    5: '连体装',
    6: '上装_长款',
    7: '裤子_长款',
}

In [9]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [10]:
chi8_map_name8 = get_list2dict(chi_label2id8.values(), chi_label2id8.keys())

### 1.Data Loader.

In [73]:
# root = Path('/nas/chenyi/benchmark/query1k_labelimg0707/')
root = Path('/nas/chenyi/datasets_nas/')

In [83]:
root_show = root /'dada_det/images/wardrobe500k/wardrobe20k_img_hard20220705'
root_show = root /'dada_det/xml/wardrobe500k/wardrobe20k_xml_hard20220705'
!ls $root_show

info_csv  xml


In [84]:
root_csv = root/'dada_det/xml/wardrobe500k/wardrobe20k_xml_hard20220705/info_csv'
!ls $root_csv

xml_clr_20220705.csv


In [20]:
xml_list = glob.glob(str(root /'xml/*/*/*.xml'))
# xml_list += glob.glob(str(root /'*/*/*.xml'))
# xml_list += glob.glob(str(root /'*/*/*/*.xml'))
len(xml_list)

578

In [21]:
# 核查xml是否有重复
xml_df = pd.DataFrame()
xml_df['path'] = xml_list
xml_df['stem'] = xml_df['path'].apply(lambda x: Path(x).stem)
len(xml_df)

578

#### csv读取和保存

In [85]:
bboxes_df = pd.read_csv(root_csv/'xml_clr_20220705.csv')

In [67]:
# !mkdir -p $root/info_csv
# bboxes_df.to_csv(root /'info_csv/query1k_xml_20220707.csv', index=False)

In [25]:
# bboxes_df = pd.read_csv(root /'info_csv/query1k_xml_20220705.csv')

#### 数据分析

In [87]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,1080,1554,裙子,6,478,1080,1129,2017022320122485968,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,4


In [92]:
bboxes_df.value_counts('labelimg'), len(bboxes_df), len(bboxes_df.drop_duplicates('stem'))

(labelimg
 上装_短款    8102
 裙子       4670
 裤子_长款    3171
 连体装      1759
 裤子_短款    1733
 包包        751
 鞋子        574
 上装_长款     211
 dtype: int64,
 20971,
 19947)

### 解析XML

#### 解析XML

In [26]:
bboxes_list = []
for p in tqdm(xml_list):
    row = parse_xml(p)
    bboxes_list += row
    # break

100%|███████████████████████████████████████████████████| 578/578 [00:00<00:00, 1072.23it/s]


In [27]:
bboxes_df = pd.DataFrame(bboxes_list)
name_stand = ['xml_path', 'width', 'height', 'labelimg', 'bx1', 'bx2','bx3', 'bx4']
bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
                          bboxes_df.columns[1]:name_stand[1],
                          bboxes_df.columns[2]:name_stand[2],
                          bboxes_df.columns[3]:name_stand[3],
                          bboxes_df.columns[4]:name_stand[4],
                          bboxes_df.columns[5]:name_stand[5],
                          bboxes_df.columns[6]:name_stand[6],
                          bboxes_df.columns[7]:name_stand[7],
                         },
                          inplace=True)

In [28]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4
0,/nas/chenyi/benchmark/query1k_labelimg0707/xml...,1862,4032,上装_短款,68,1268,1862,2967


In [29]:
chi_label2id8

{0: '鞋子',
 1: '包包',
 2: '上装_短款',
 3: '裤子_短款',
 4: '裙子',
 5: '连体装',
 6: '上装_长款',
 7: '裤子_长款'}

In [30]:
bboxes_df.value_counts('labelimg'), len(bboxes_df), len(bboxes_df.value_counts('labelimg').index)

(labelimg
 上装_短款    161
 鞋子       138
 裙子        83
 裤子_长款     79
 裤子_短款     67
 包包        64
 上装_长款     53
 连体装       53
 dtype: int64,
 698,
 8)

In [31]:
bboxes_df['stem'] = bboxes_df['xml_path'].apply(lambda x: Path(x).stem)

In [32]:
chi8_map_name8

{'鞋子': 0,
 '包包': 1,
 '上装_短款': 2,
 '裤子_短款': 3,
 '裙子': 4,
 '连体装': 5,
 '上装_长款': 6,
 '裤子_长款': 7}

In [33]:
# bboxes_df['label86'] = bboxes_df['labelimg'].map(chi8_map_name8)

##### 修正label

In [34]:
len(bboxes_df.value_counts('labelimg'))

8

In [35]:
# bboxes_df[bboxes_df['labelimg']=='上装短款'].values

In [36]:
# bboxes_df.loc[bboxes_df[bboxes_df['labelimg']=='上装短款'].index, 'labelimg'] = '上装_短款'

#### 获取对应图片信息

In [37]:
!ls $root/images

anomaly  object


In [68]:
img_list = glob.glob(str(root /'images/*/*/*/*'))
img_df = pd.DataFrame()
img_df['path'] = img_list
len(img_df)

934

In [69]:
img_df['stem'] = img_df['path'].apply(lambda x: Path(x).stem)

In [70]:
img_df['parts3'] = img_df['path'].apply(lambda x: Path(x).parts[-3])

In [71]:
img_df.value_counts('parts3')

parts3
easy        387
anomaly     353
hard        107
rotation     87
dtype: int64

In [72]:
len(img_df)

934

##### 获取bbox对应的图片

In [43]:
# xml是否都有对应的图片
cross_df = img_df[img_df['stem'].isin(bboxes_df['stem'])]
len(cross_df)

578

In [45]:
stem2path = get_list2dict(cross_df['stem'].values, cross_df['path'].values)

In [46]:
bboxes_df['img_sp'] = bboxes_df['stem'].map(stem2path)

In [47]:
bboxes_clr_df = bboxes_df[~bboxes_df['img_sp'].isna()]

In [48]:
len(bboxes_df), len(bboxes_clr_df)

(698, 698)

##### 获取图片的宽和高

In [49]:
bboxes_df = bboxes_clr_df

In [50]:
w_list = []
h_list = []
for ind,row in tqdm(bboxes_df.iterrows()):
    img_sp = row['img_sp']
    img = Image.open(img_sp)
    w, h = img.size
    w_list.append(w)
    h_list.append(h)
    # break

698it [00:01, 551.81it/s] 


In [51]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp
0,/nas/chenyi/benchmark/query1k_labelimg0707/xml...,1862,4032,上装_短款,68,1268,1862,2967,c7ebf676-d697-42df-b99a-17a9f6337425,/nas/chenyi/benchmark/query1k_labelimg0707/ima...


In [52]:
bboxes_df['width'] = w_list
bboxes_df['height'] = h_list

#### label处理

In [53]:
chi8_map_name8

{'鞋子': 0,
 '包包': 1,
 '上装_短款': 2,
 '裤子_短款': 3,
 '裙子': 4,
 '连体装': 5,
 '上装_长款': 6,
 '裤子_长款': 7}

In [54]:
bboxes_df['label86'] = bboxes_df['labelimg'].map(chi8_map_name8)

In [55]:
bboxes_df['label_model'] = bboxes_df['label86'].values
bboxes_df.loc[bboxes_df['label_model']==6, 'label_model'] = 2
bboxes_df.loc[bboxes_df['label_model']==7, 'label_model'] = 3

In [56]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86,label_model
0,/nas/chenyi/benchmark/query1k_labelimg0707/xml...,1862,4032,上装_短款,68,1268,1862,2967,c7ebf676-d697-42df-b99a-17a9f6337425,/nas/chenyi/benchmark/query1k_labelimg0707/ima...,2,2


In [57]:
bboxes_df.value_counts('label86').sort_values()

label86
5     53
6     53
1     64
3     67
7     79
4     83
0    138
2    161
dtype: int64

In [58]:
bboxes_df.value_counts('labelimg').sort_values()

labelimg
上装_长款     53
连体装       53
包包        64
裤子_短款     67
裤子_长款     79
裙子        83
鞋子       138
上装_短款    161
dtype: int64

##### 统计人员的bbox

In [59]:
# df['man'] = df[int(df.columns[0])].apply(lambda x: re.findall('wholebody0609_xml/.*?_(.*?)/', x)[0])
# df.value_counts('man')

### one object.

In [60]:
eng2num = {'shoe': 0, 
         'upperbody_short':2,
         'bag':1, 
         'lowbody_long':7, 
         'upperbody_long':6,
         'skirt':4, 
         'wholebody':5, 
         'lowbody_short':3}

In [61]:
eng2num_hard = {'0_shoe': 0, 
                 '1_bag':1, 
         '2_upperbody_short':2,
         '3_lowbody_short':3, 
         '4_skirt':4, 
         '5_wholebody':5, 
         '6_upperbody_long':6,
         '7_lowbody_long':7}

In [62]:
bboxes_df['cls_folder'] = bboxes_df['img_sp'].apply(lambda x: Path(x).parts[-2])

In [63]:
# bboxes_df['label_folder'] = bboxes_df['cls_folder'].map(eng2num)
bboxes_df['label_folder'] = bboxes_df['cls_folder'].map(eng2num_hard)

In [64]:
bboxes_eq_df = bboxes_df[bboxes_df['label_folder']==bboxes_df['label86']]

In [65]:
bboxes_df['label_query'] = bboxes_df['label_folder']==bboxes_df['label86']

In [66]:
len(bboxes_eq_df)

618

### 数据展示

In [47]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/dada_det

/home/chenyi/workspace/dataset/data/dada_det
├── hard_sample_0610
├── __MACOSX
├── shoe_bag11k_may
├── shoeBag5k0515
└── wholebody0601_all

5 directories


In [25]:
ROOT = Path('/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/')

#### 处理打标标签

##### 构建中文标签和业务id映射

In [45]:
# counter = df.value_counts(df.columns[3])
# name_list = [item[0] for item in list(counter.index)]
# name_df = pd.DataFrame()
# name_df['name'] = name_list

In [121]:
import json
with open('../../../data/dict/name_chi2l1.json', 'r') as f:
    name_chi2l1 = json.load(f)

In [122]:
df['label6'] = df[df.columns[3]].map(name_chi2l1)

In [123]:
df.value_counts('label6')

label6
4    27
7    19
6    8 
8    4 
5    2 
dtype: int64

In [115]:
# 增加多出的中文标签映射
# name_chi2l1['皮衣外套'] = 4

In [116]:
import json
with open('../../../data/dict/name_chi2l1.json', 'w') as f:
    json.dump(name_chi2l1, f)

In [124]:
model_label_map

{0: 10, 1: 8, 2: 4, 3: 5, 4: 6, 5: 7}

In [126]:
l1_map_model = get_list2dict(model_label_map.values(), model_label_map.keys())
df['label_model'] = df['label6'].map(l1_map_model)

##### 存储目标index

In [ ]:
# df = pd.read_csv('../data/wh.csv')

In [ ]:
!ls /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
!mkdir -p /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed')

In [ ]:
df.head(2).values

In [ ]:
df['img_tp'] = df['img_sp'].apply(lambda x:'{0}/{1}/{2}'.format(draw_root, Path(x).parts[-2], Path(x).name))

In [ ]:
sys.path.append('../../../')
from utils.plots.plots import draw_bbox
import cv2
from PIL import Image

In [ ]:
# error_p_list = []
# for ind, row in tqdm(df.iterrows()):
#     row_info = row.loc[df.columns[:8]].values
#     w,h = row_info[1], row_info[2]
#     name = row.loc[['label_model']].values[0]
#     bbox = list(row_info[4:8])
#     bbox.insert(0,name)
#     tp = Path(row.loc[['img_tp']].values[0])
#     if not tp.parent.is_dir():
#         tp.parent.mkdir()
#     rp = tp if tp.is_file() else row.loc[['img_sp']].values[0]
#     img = cv2.imread(str(rp))
#     img = draw_bbox(img, bbox)
#     try:
#         cv2.imwrite(str(tp), img)
#     except:
#         error_p_list.append(tp)
#     # break

In [ ]:
error_p_list

In [ ]:
error_df = pd.DataFrame()
error_df['path'] = error_p_list
error_df['stem'] = error_df['path'].apply(lambda x: Path(x).stem)

In [ ]:
df.head()

In [ ]:
df = df[~df['stem'].isin(error_df['stem'])]

In [ ]:
# df.to_csv('../data/wholebody5k0610/wholebody5k_july.csv', index=False)

#### Display Html

In [ ]:
sys.path.append('../../../')
from utils.display.html import *
from IPython.display import HTML
from io import BytesIO

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed/')

In [ ]:
img_list = glob.glob(str(draw_root/'*/*.jpg'))
len(img_list), img_list[-1]

In [ ]:
# img_list = glob.glob(draw_root + '*.jpg')
df_drawed = pd.DataFrame()
df_drawed['path'] = img_list[-100:]

In [ ]:
size = 500
df_drawed['res'] = df_drawed['path'].map(lambda x: get_thumbnail(x, size))

In [ ]:
f = df_drawed[['res', 'path']].to_html(formatters={'res': image_formatter}, escape=False)

In [ ]:
HTML(f)

### 解析json文件

In [ ]:
json_list = glob.glob('/home/chenyi/workspace/dataset/haowei34k_labelImg/xml_label/*/*.json')

In [ ]:
json_df = pd.DataFrame()
json_df['spath'] = json_list

In [ ]:
xml_dir = Path('/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/')

In [ ]:
json_df['tpath'] = json_df['spath'].apply(lambda x:
                                         '{0}/{1}'.format(xml_dir, re.findall(r'/[0-9]{1,3}/(.*?.json)',x)[0]))

In [ ]:
sys.path.append('/home/chenyi/workspace/myPyModule99/data_format/json_to_xml')
from json_to_xml.main import *

In [ ]:
root_json_dir = xml_dir
root_save_xml_dir = '/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/xml'

In [ ]:
import json
for json_filename in tqdm(os.listdir(root_json_dir)):
    json_path = os.path.join(root_json_dir, json_filename)
    json_data = json.load(open(json_path))
    break

In [ ]:
json_df['path'] = glob.glob(str(root_json_dir/'*.json'))

In [ ]:
json_df['fname'] =  json_df['path'].apply(lambda x: re.findall(r'/json2xml/(.*?).json',x)[0])

In [ ]:
json_labels = {}
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    json_labels[fname] = ''
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        json_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
            classes_name[name], xmin, ymin, xmax, ymax)

In [ ]:
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

In [ ]:
json_bboxes = pd.DataFrame()

In [ ]:
json_bboxes['fname'] = fname_list
json_bboxes['label'] = labels_list
json_bboxes['xmin'] = xmin_list
json_bboxes['ymin'] = ymin_list
json_bboxes['xmax'] = xmax_list
json_bboxes['ymax'] = ymax_list

In [ ]:
json_bboxes.head()